### Hiperweb, Browser based competitor Web Scraping Template

### installs

In [ ]:
#Install Modules
! pip install bs4
! pip install requests
! pip install mechanize
! pip install http
! pip install json
! pip install pandas


### imports

In [1]:
from bs4 import BeautifulSoup as bs
from base64 import b64encode
import requests as req
import mechanize as mz
from urllib.request import Request, urlopen
import http.cookiejar as ck
import json
import pandas as pd

In [11]:
# Box of Variables. We like variables. We have our targeted webpage, our username and password, and some abstractions
# CookieJar allows us to transfer cookies between our python webpages.
# Mechanize allows us to automate our web navigation: to log in, move between pages, etc.

client_name = '###'
username = "###"
password = "###"

#client_name = '###'
#username = '###'
#password = '###'
bl_class = {}

## NOTICING 2 VARIANTS: https://{CLIENT}.hiperweb.com/
## vs.                  https://financials.hiperweb.com/{CLIENT}/

bl_class_url = f'https://{client_name}.hiperweb.com/BL/Class'
get_bl_classes_url = f'https://{client_name}.hiperweb.com/BL/GetClassesDS?BLClassGrid-mode=insert'
get_bl_subclass_url = '' # f'https://{client_name}.hiperweb.com/BL/GetSubClasses/{bl_class}?BLClassGrid-sort=ClassID-asc'
get_bl_listing_url = f'https://{client_name}.hiperweb.com/BL'


#bl_class_url = f'https://financials.hiperweb.com/{client_name}BL/Class'
#get_bl_classes_url = f'https://financials.hiperweb.com/{client_name}/BL/GetClassesDS?BLClassGrid-mode=insert'
#get_bl_subclass_url = f'https://financials.hiperweb.com/{client_name}/BL/GetSubClasses/{bl_class}?BLClassGrid-sort=ClassID-asc'
#get_business_url = f'https://financials.hiperweb.com/{client_name}/BL/GetBusinesses?BLCustomerGrid-mode=insert'
#get_license_url = f'https://financials.hiperweb.com/{client_name}/BL/BL_Read?BLGrid-sort=BLLicenseID-asc'

classes_export = "classes_export.xlsx"
subclass_export = "subclass_export.xlsx"
business_license_export = "business_license_export.xlsx"
license_export = "license_export.xlsx"
business_export = "business_export.xlsx"



In [12]:
cj = ck.CookieJar()
br = mz.Browser()
br.set_cookiejar(cj)

br.set_handle_equiv(True)
br.set_handle_redirect(True)
br.set_handle_robots(False)

In [13]:
br.open(bl_class_url)

#Identifies the first form (0 is the lowest)
br.select_form(nr=0)

#Shows us our session response, prints forms.
forms = br.forms()
print(forms)

#For the given form, inputs our variables into the "name=" portion of the field.
br.form['UserName'] = username
br.form['Password'] = password

#Submit request.
br.submit()

#Note at this point we're at the main page for Fitzgerald.
print(br.geturl())

#From this point on we're in the main page. In the below text box, notice the first instance of our forms has a different object
# @. We've hit the main page of fitzgerald.hiperweb.com/.
forms = br.forms()
print(forms)


https://fitzgerald.hiperweb.com/


### Bypass login form if setting cookies

In [5]:

br.set_simple_cookie('__RequestVerificationToken_L0FzaGJ1cm41',
              '7AX1jRt7MtxYYFBIY06_bWCwGPgyjJ4Rosd8RzhMR4rsABHysLXMokAROkyhlniHVwgYgS1aI6mDlQ7BN0Lpnh1HQHMoFVZ9ihYpC5MYV2E1',
              'financials.hiperweb.com')
br.set_simple_cookie('.ASPXAUTH', 
                'ED251DFF959292770BDA9DDBC20ED1F876E1401A59FF096B67606B68A43523A54F829455C8133A54BA856E0179D534376F9704FEA6D672C9DE17FB8C77AE7A6F18A26F9E5D7A5E855119C5221E0950333F2FEC1B4065C4CBCFA5239A4E8DC9DA',
                'financials.hiperweb.com')

In [14]:
#From manually searching through the webpage on my own Chrome client, I'm looking for href="/BL/GetClassesDS?BLClassGrid-mode-insert"
#Following that, below is the URL for the above source.

#Open that body as rsp, or response, to save the response.
rsp = br.open(get_bl_classes_url)

body = rsp.read() #Save the result to a variable for abstraction.

# Further abstraction but we're taking the result and loading it out of the b-type json.
result = json.loads(body)
data = result['Data']
#Shows us one example of what we're looking at pulling down.

#Storing result as 'Data'

df = pd.read_json(json.dumps(data))
df.to_excel(classes_export, index=False)

print(df)

    ClassID                                          ClassDesc  BLClassCount  \
0         1                                  Service Providers           285   
1         2           RETAIL SHOPS / STATION / STORES / SUPPLY           162   
2         3                       Entertainment / Recreational            24   
3         4                         Restaurants / Food Vendors           102   
4         5                                               AUTO            68   
5         6                                    Assisted Living             5   
6         7                       Clothing & Department Stores            60   
7         8                         INDUSTRIES / MANUFACTURING            42   
8         9                                          WHOLESALE             8   
9        13                            Motel / Hotel / Lodging            17   
10       14                                        Contractors           141   
11       16                             

In [16]:
# This looks at our data frame and pulls the Class IDs.
# From there, we assign the Class IDs (plural, per the for loop) to bl_class, each time it's run.
# Targeting the subclass_url, it replaces the query going to the website's database at the {bl_class} portion.
# That subclass_url pulls up another json of the subclass tables, comprised of SubclassID, ClassID, SubClass, SubClassDesc, and
# various fees. From there, we store the results, open them, and read them out.
# the print(d) line is unnecessary, hence why it is commented out, but lets us show the 

df = pd.DataFrame()

for d in data: 
    #print(d)
    bl_class = d['ClassID']
    
    subclass_url = f'http://{client_name}.hiperweb.com/BL/GetSubClasses/{bl_class}?BLClassGrid-sort=ClassID-asc'
    #subclass_url = f'http://financials.hiperweb.com/{client_name}/BL/GetSubClasses/{bl_class}?BLClassGrid-sort=ClassID-asc'
    
    # print(subclass_url)
    subclass_rsp = br.open(subclass_url)
    subclass_body = subclass_rsp.read()
    subclass_result = json.loads(subclass_body)
    subclass_data = subclass_result['Data']

    temp_df = pd.read_json(json.dumps(subclass_data))
    df = pd.concat([df, temp_df])
    
df.to_excel(subclass_export, index=False)
print(df)

    SubClassID  ClassID SubClass  \
0            1        1        A   
1            2        1        B   
2            3        1        C   
3            4        1        D   
4            5        1        E   
..         ...      ...      ...   
9          475       35        J   
10         476       35        K   
0          452       36        A   
0          453       50        A   
1          454       50        B   

                                         SubClassDesc  SubLicenseFee  \
0                             ADVERTISING/PROMOTIONAL            0.0   
1                                     ANIMAL BOARDING            0.0   
2                          ANIMAL BOARDING & GROOMING            0.0   
3                                      ANIMAL BREEDER            0.0   
4                                     ANIMAL GROOMING            0.0   
..                                                ...            ...   
9                         SPECIAL EVENT - BEER & WINE          

### Export Data for Business Licenses

In [17]:
license_ids = [str(i) for i in range(1,10301)]

In [19]:

license_list = []
for license_num in license_ids:

    try:

        #license_num = '1225'
        #bl_license_url = f'https://financials.hiperweb.com/{client_name}/BL/License/{license_num}'
        bl_license_url = f'https://{client_name}.hiperweb.com/BL/License/{license_num}'

        rsp = br.open(bl_license_url)


        body = rsp.read() #Save the result to a variable for abstraction.
        soup = bs(body, 'lxml')

        field_map = {}
        for el in soup.find_all('input'):
            try:
                el_name = el['name']
                el_value = el['value']
                field_map[el_name] = el_value
                # print(f'{ el_name} : {el_value} ')
            except Exception as e:
                pass

        i=0
        for el in soup.find_all("div", {"class": "editor-label"}):
            try:
                if not el.style:
                    i+=1
                    el_text = el.text.replace("\n","").replace("\r","").strip()
                    field_map[f'label{i}'] = el_text
                    # print(f'field{i} : {el.text}')
            except Exception as e:
                pass

        i=0
        for el in soup.find_all("div", {"class": "editor-field"}):
            try:
                if not el.style:
                    i+=1
                    el_text = el.text.replace("\n","").replace("\r","").strip()
                    el_next = el.next.replace("\n","").replace("\r","").strip()
   
                    if el_text != el_next:
                        el_text = el_next + "|" + el_text.replace(el_next,"")
                        print("")

                    field_map[f'field{i}'] = el_text
                    # print(f'field{i} : {el.text}')
            except Exception as e:
                pass

        license_list.append(field_map)

    except Exception as e:
        print(e)
        print(license_num)

df = pd.DataFrame.from_records(license_list)
df.to_excel(business_license_export, index=False)
print(df)

HTTP Error 404: Not Found
1
HTTP Error 404: Not Found
2
HTTP Error 404: Not Found
3
HTTP Error 404: Not Found
4
HTTP Error 404: Not Found
5
HTTP Error 404: Not Found
6
HTTP Error 404: Not Found
7
HTTP Error 404: Not Found
8
HTTP Error 404: Not Found
9



HTTP Error 404: Not Found
11
HTTP Error 404: Not Found
12
HTTP Error 404: Not Found
13
HTTP Error 404: Not Found
14
HTTP Error 404: Not Found
15
HTTP Error 404: Not Found
16
HTTP Error 404: Not Found
17
HTTP Error 404: Not Found
18
HTTP Error 404: Not Found
19
HTTP Error 404: Not Found
20
HTTP Error 404: Not Found
21
HTTP Error 404: Not Found
22
HTTP Error 404: Not Found
23
HTTP Error 404: Not Found
24
HTTP Error 404: Not Found
25
HTTP Error 404: Not Found
26
HTTP Error 404: Not Found
27
HTTP Error 404: Not Found
28
HTTP Error 404: Not Found
29
HTTP Error 404: Not Found
30
HTTP Error 404: Not Found
31
HTTP Error 404: Not Found
32
HTTP Error 404: Not Found
33
HTTP Error 404: Not Found
34
HTTP Error 404: Not Found
35
HTTP Error 404: Not 

HTTP Error 404: Not Found
289



HTTP Error 404: Not Found
291
HTTP Error 404: Not Found
292
HTTP Error 404: Not Found
293
HTTP Error 404: Not Found
294
HTTP Error 404: Not Found
295
HTTP Error 404: Not Found
296
HTTP Error 404: Not Found
297
HTTP Error 404: Not Found
298
HTTP Error 404: Not Found
299
HTTP Error 404: Not Found
300
HTTP Error 404: Not Found
301
HTTP Error 404: Not Found
302
HTTP Error 404: Not Found
303
HTTP Error 404: Not Found
304
HTTP Error 404: Not Found
305
HTTP Error 404: Not Found
306
HTTP Error 404: Not Found
307
HTTP Error 404: Not Found
308
HTTP Error 404: Not Found
309
HTTP Error 404: Not Found
311
HTTP Error 404: Not Found
312
HTTP Error 404: Not Found
313
HTTP Error 404: Not Found
314
HTTP Error 404: Not Found
315
HTTP Error 404: Not Found
316
HTTP Error 404: Not Found
317
HTTP Error 404: Not Found
318
HTTP Error 404: Not Found
319
HTTP Error 404: Not Found
320
HTTP Error 404: Not Found
321
HTTP Error 404: Not Found
322
HTTP Error 404: Not Found
323
HTTP Er




HTTP Error 404: Not Found
571
HTTP Error 404: Not Found
572
HTTP Error 404: Not Found
573
HTTP Error 404: Not Found
574
HTTP Error 404: Not Found
575
HTTP Error 404: Not Found
576
HTTP Error 404: Not Found
577
HTTP Error 404: Not Found
578
HTTP Error 404: Not Found
579
HTTP Error 404: Not Found
580
HTTP Error 404: Not Found
581
HTTP Error 404: Not Found
582
HTTP Error 404: Not Found
583
HTTP Error 404: Not Found
584
HTTP Error 404: Not Found
585
HTTP Error 404: Not Found
586
HTTP Error 404: Not Found
587
HTTP Error 404: Not Found
588
HTTP Error 404: Not Found
589
HTTP Error 404: Not Found
590
HTTP Error 404: Not Found
591
HTTP Error 404: Not Found
592
HTTP Error 404: Not Found
593
HTTP Error 404: Not Found
594
HTTP Error 404: Not Found
595
HTTP Error 404: Not Found
596
HTTP Error 404: Not Found
597
HTTP Error 404: Not Found
598
HTTP Error 404: Not Found
599
HTTP Error 404: Not Found
600
HTTP Error 404: Not Found
601
HTTP Error 404: Not Found
602
HTTP Error 404: Not Found
603
HTTP Er

HTTP Error 404: Not Found
850
HTTP Error 404: Not Found
851
HTTP Error 404: Not Found
852
HTTP Error 404: Not Found
853
HTTP Error 404: Not Found
854
HTTP Error 404: Not Found
855
HTTP Error 404: Not Found
856
HTTP Error 404: Not Found
857
HTTP Error 404: Not Found
858
HTTP Error 404: Not Found
859
HTTP Error 404: Not Found
860
HTTP Error 404: Not Found
861
HTTP Error 404: Not Found
862
HTTP Error 404: Not Found
863
HTTP Error 404: Not Found
864
HTTP Error 404: Not Found
865
HTTP Error 404: Not Found
866
HTTP Error 404: Not Found
867
HTTP Error 404: Not Found
868
HTTP Error 404: Not Found
869
HTTP Error 404: Not Found
870
HTTP Error 404: Not Found
871
HTTP Error 404: Not Found
872
HTTP Error 404: Not Found
873
HTTP Error 404: Not Found
874
HTTP Error 404: Not Found
875
HTTP Error 404: Not Found
876
HTTP Error 404: Not Found
877
HTTP Error 404: Not Found
878
HTTP Error 404: Not Found
879
HTTP Error 404: Not Found
880
HTTP Error 404: Not Found
881
HTTP Error 404: Not Found
882
HTTP Error

HTTP Error 404: Not Found
1128
HTTP Error 404: Not Found
1129



HTTP Error 404: Not Found
1131
HTTP Error 404: Not Found
1132
HTTP Error 404: Not Found
1133
HTTP Error 404: Not Found
1134
HTTP Error 404: Not Found
1135
HTTP Error 404: Not Found
1136
HTTP Error 404: Not Found
1137
HTTP Error 404: Not Found
1138
HTTP Error 404: Not Found
1139
HTTP Error 404: Not Found
1140
HTTP Error 404: Not Found
1141
HTTP Error 404: Not Found
1142
HTTP Error 404: Not Found
1143
HTTP Error 404: Not Found
1144
HTTP Error 404: Not Found
1145
HTTP Error 404: Not Found
1146
HTTP Error 404: Not Found
1147
HTTP Error 404: Not Found
1148
HTTP Error 404: Not Found
1149
HTTP Error 404: Not Found
1150
HTTP Error 404: Not Found
1151
HTTP Error 404: Not Found
1152
HTTP Error 404: Not Found
1153
HTTP Error 404: Not Found
1154
HTTP Error 404: Not Found
1155
HTTP Error 404: Not Found
1156
HTTP Error 404: Not Found
1157
HTTP Error 404: Not Found
1158
HTTP Error 404: Not Found
1159



HTTP Error 404: Not Found
1161
HT

...

HTTP Error 404: Not Found
8250
HTTP Error 404: Not Found
8251
HTTP Error 404: Not Found
8252
HTTP Error 404: Not Found
8253
HTTP Error 404: Not Found
8254
HTTP Error 404: Not Found
8255
HTTP Error 404: Not Found
8256
HTTP Error 404: Not Found
8257
HTTP Error 404: Not Found
8258
HTTP Error 404: Not Found
8259
HTTP Error 404: Not Found
8260
HTTP Error 404: Not Found
8261
HTTP Error 404: Not Found
8262
HTTP Error 404: Not Found
8263
HTTP Error 404: Not Found
8264
HTTP Error 404: Not Found
8265
HTTP Error 404: Not Found
8266
HTTP Error 404: Not Found
8267
HTTP Error 404: Not Found
8268
HTTP Error 404: Not Found
8269
HTTP Error 404: Not Found
8270
HTTP Error 404: Not Found
8271
HTTP Error 404: Not Found
8272
HTTP Error 404: Not Found
8273
HTTP Error 404: Not Found
8274
HTTP Error 404: Not Found
8275



HTTP Error 404: Not Found
8277



HTTP Error 404: Not Found
8279


HTTP Error 404: Not Found
8281




HTTP Error 404: Not Found
8284












HTTP Error 404: Not Found
8289

















HTTP Error 404: Not Found
8577
HTTP Error 404: Not Found
8578
HTTP Error 404: Not Found
8579
HTTP Error 404: Not Found
8580
HTTP Error 404: Not Found
8581
HTTP Error 404: Not Found
8582
HTTP Error 404: Not Found
8583
HTTP Error 404: Not Found
8584



HTTP Error 404: Not Found
8586
HTTP Error 404: Not Found
8587
HTTP Error 404: Not Found
8588






HTTP Error 404: Not Found
8591
HTTP Error 404: Not Found
8592



HTTP Error 404: Not Found
8594
HTTP Error 404: Not Found
8595
HTTP Error 404: Not Found
8596
HTTP Error 404: Not Found
8597
HTTP Error 404: Not Found
8598
HTTP Error 404: Not Found
8599






HTTP Error 404: Not Found
8602


HTTP Error 404: Not Found
8604
HTTP Error 404: Not Found
8605
HTTP Error 404: Not Found
8606


HTTP Error 404: Not Found
8608
HTTP Error 404: Not Found
8609



HTTP Error 404: Not Found
8611
HTTP Error 404: Not Found
8612
HTTP Error 404: Not Found
8613
HTTP Error 404: Not Found
8614
HTTP Error 404: Not Found
8615
HTTP Error 404: Not Found
8616
HTTP Error 404







HTTP Error 404: Not Found
8946






HTTP Error 404: Not Found
8949





HTTP Error 404: Not Found
8952
HTTP Error 404: Not Found
8953


HTTP Error 404: Not Found
8955
HTTP Error 404: Not Found
8956
HTTP Error 404: Not Found
8957












HTTP Error 404: Not Found
8962



HTTP Error 404: Not Found
8964



HTTP Error 404: Not Found
8966
HTTP Error 404: Not Found
8967






HTTP Error 404: Not Found
8970
HTTP Error 404: Not Found
8971






HTTP Error 404: Not Found
8974
HTTP Error 404: Not Found
8975
HTTP Error 404: Not Found
8976
HTTP Error 404: Not Found
8977
HTTP Error 404: Not Found
8978
HTTP Error 404: Not Found
8979
HTTP Error 404: Not Found
8980
HTTP Error 404: Not Found
8981
HTTP Error 404: Not Found
8982
HTTP Error 404: Not Found
8983
HTTP Error 404: Not Found
8984
HTTP Error 404: Not Found
8985
HTTP Error 404: Not Found
8986
HTTP Error 404: Not Found
8987
HTTP Error 404: Not Found
8988
HTTP Error 404: Not Found
8989
HTTP Error 404: Not Found
8990
HTTP Error 404: Not F




HTTP Error 404: Not Found
9362
HTTP Error 404: Not Found
9363
HTTP Error 404: Not Found
9364
HTTP Error 404: Not Found
9365
HTTP Error 404: Not Found
9366
HTTP Error 404: Not Found
9367
HTTP Error 404: Not Found
9368
HTTP Error 404: Not Found
9369
HTTP Error 404: Not Found
9370
HTTP Error 404: Not Found
9371






HTTP Error 404: Not Found
9374
HTTP Error 404: Not Found
9375
HTTP Error 404: Not Found
9376



HTTP Error 404: Not Found
9378






HTTP Error 404: Not Found
9381
HTTP Error 404: Not Found
9382
HTTP Error 404: Not Found
9383
HTTP Error 404: Not Found
9384
HTTP Error 404: Not Found
9385
HTTP Error 404: Not Found
9386
HTTP Error 404: Not Found
9387
HTTP Error 404: Not Found
9388
HTTP Error 404: Not Found
9389
HTTP Error 404: Not Found
9390


HTTP Error 404: Not Found
9392
HTTP Error 404: Not Found
9393
HTTP Error 404: Not Found
9394
HTTP Error 404: Not Found
9395
HTTP Error 404: Not Found
9396
HTTP Error 404: Not Found
9397



HTTP Error 404: Not Found
9399
HTTP Error 404: 













HTTP Error 404: Not Found
10196
HTTP Error 404: Not Found
10197
HTTP Error 404: Not Found
10198
HTTP Error 404: Not Found
10199
HTTP Error 404: Not Found
10200
HTTP Error 404: Not Found
10201
HTTP Error 404: Not Found
10202
HTTP Error 404: Not Found
10203
HTTP Error 404: Not Found
10204
HTTP Error 404: Not Found
10205
HTTP Error 404: Not Found
10206
HTTP Error 404: Not Found
10207
HTTP Error 404: Not Found
10208
HTTP Error 404: Not Found
10209
HTTP Error 404: Not Found
10210
HTTP Error 404: Not Found
10211
HTTP Error 404: Not Found
10212
HTTP Error 404: Not Found
10213
HTTP Error 404: Not Found
10214
HTTP Error 404: Not Found
10215
HTTP Error 404: Not Found
10216
HTTP Error 404: Not Found
10217
HTTP Error 404: Not Found
10218
HTTP Error 404: Not Found
10219
HTTP Error 404: Not Found
10220
HTTP Error 404: Not Found
10221
HTTP Error 404: Not Found
10222
HTTP Error 404: Not Found
10223
HTTP Error 404: Not Found
10224
HTTP Error 404: Not Found
10225
HTTP Error 404: Not Found
10

### Generic function to dump JSON/application data from web URL into an excel/tabulated file

In [20]:
def dump_url_to_excel(url, file_name):
    #Open that body as rsp, or response, to save the response.
    rsp = br.open(url)
    body = rsp.read() #Save the result to a variable for abstraction.

    # Further abstraction but we're taking the result and loading it out of the b-type json.
    result = json.loads(body)
    data = result['Data']
    #Shows us one example of what we're looking at pulling down.

    #Storing result as 'Data'

    df = pd.read_json(json.dumps(data))
    df.to_excel(file_name, index=False)
    print(df)

### Exports

In [26]:
#dump_url_to_excel(get_business_url, business_export)
#dump_url_to_excel(get_bl_listing_url, business_export)
dump_url_to_excel(bl_license_url, license_export)
dump_url_to_excel(get_bl_classes_url, classes_export)


httperror_seek_wrapper: HTTP Error 404: Not Found